In [ ]:
# Kiểm tra setup
try:
    from shared_variables import *
except:
    # Tải các biến và hàm từ notebook trước
    %run "1_Setup_and_Utils.ipynb"
    %run "2_Image_Processing.ipynb"

# Định nghĩa lớp SignLanguageModel
class SignLanguageModel:
    def __init__(self, model_path=None):
        self.model_path = model_path
        self.model = None
        self.labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                       'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
                       'space', 'delete', 'nothing']
        
        if self.model_path:
            self.load_model()
        else:
            self._create_model()

    def _create_model(self):
        """Create a CNN model for sign language recognition"""
        model = tf.keras.Sequential([
            # Lớp Conv2D đầu tiên
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
            tf.keras.layers.MaxPooling2D((2, 2)),
            
            # Lớp Conv2D thứ hai
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            
            # Lớp Conv2D thứ ba
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            
            # Flatten và các lớp fully connected
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(len(self.labels), activation='softmax')
        ])
        
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        
        self.model = model
        print("Created a new model. Note: This model is untrained.")

    def load_model(self):
        try:
            self.model = tf.keras.models.load_model(self.model_path)
            print(f"Model loaded successfully from {self.model_path}")
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Creating a new model instead...")
            self._create_model()

    def preprocess_image(self, image):
        """Preprocess the input image for prediction"""
        # Lấy hàm resize_image từ shared variables
        resize_image = load_variable('resize_image')
        if resize_image is None:
            # Fallback nếu không tìm thấy hàm
            from 2_Image_Processing import resize_image
        
        # Convert OpenCV image to PIL Image
        if len(image.shape) == 3:  # Color image
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        else:  # Already grayscale
            image = Image.fromarray(image)
        
        # Resize image
        image = resize_image(image, (64, 64), preserve_aspect_ratio=True)
        
        # Convert to grayscale if needed
        if isinstance(image, Image.Image):
            if image.mode != 'L':
                image = image.convert('L')
            img_array = np.array(image)
        else:
            if len(image.shape) == 3:
                img_array = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            else:
                img_array = image
        
        # Normalize and reshape
        img_array = img_array.astype('float32') / 255.0
        img_array = img_array.reshape(1, 64, 64, 1)
        
        return img_array

    def predict_sign(self, image):
        """Predict the sign from an image"""
        # Ensure image is grayscale
        if len(image.shape) == 3:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray_image = image
            
        # Check if the image has enough variation to be a hand
        std_dev = np.std(gray_image)
        if std_dev < 20:  # If low variation, likely no hand
            return "?", 0.0

        # Process image
        processed_image = self.preprocess_image(gray_image)
        
        # Make prediction
        predictions = self.model.predict(processed_image, verbose=0)
        
        # Get the index of the highest confidence prediction
        predicted_index = np.argmax(predictions[0])
        confidence = predictions[0][predicted_index]
        
        # Map the index to the sign
        if predicted_index < 26:  # A-Z
            predicted_sign = chr(65 + predicted_index)
        else:
            special_classes = {26: "space", 27: "delete", 28: "nothing"}
            predicted_sign = special_classes.get(predicted_index, "?")
        
        return predicted_sign, confidence

# Lưu model class để chia sẻ
save_variable(SignLanguageModel, 'SignLanguageModel')

# Hiển thị kiến trúc mô hình để kiểm tra
model = SignLanguageModel()
model.model.summary()

# Tùy chọn lưu model để chia sẻ
save_variable(model, 'model_instance')